## Stream attributes using Signals

This notebook creates a new attribute group using the SDK that will be computed using stream processing.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

# Installation and setup

In [ ]:
%pip install snowplow-signals

In [ ]:
from snowplow_signals import Signals
import os

try:
    from google.colab import userdata
    sp_signals = Signals(
            api_url=userdata.get('SP_API_URL'),
            api_key=userdata.get('SP_API_KEY'),
            api_key_id=userdata.get('SP_API_KEY_ID'),
            org_id=userdata.get('SP_ORG_ID'),
        )
except ImportError:
    from dotenv import load_dotenv
    load_dotenv()
    sp_signals = Signals(
        api_url=os.environ['SP_API_URL'],
        api_key=os.environ['SP_API_KEY'],
        api_key_id=os.environ['SP_API_KEY_ID'],
        org_id=os.environ['SP_ORG_ID'],
)

### Define a new attribute

This block creates a single attribute definition including the logic how it should be calculated (it's filters and aggregation).

The attribute calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import Attribute, Criteria, Criterion, Event, EntityProperty, EventProperty

products_added_to_cart = Attribute(
    name="products_added_to_cart",
    type="string_list",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow.ecommerce",
            name="snowplow_ecommerce_action",
            version="1-0-2",
        )
    ],
    aggregation="unique_list",
    property=EntityProperty(
        vendor="com.snowplowanalytics.snowplow.ecommerce",
        name="product",
        major_version=1,
        path="name",
    ),
    criteria=Criteria(
        all=[
            Criterion.eq(
                EventProperty(
                    vendor="com.snowplowanalytics.snowplow.ecommerce",
                    name="snowplow_ecommerce_action",
                    major_version=1,
                    path="type",
                ),
                "add_to_cart",
            ),
        ],
    ),
)

### Wrapping the attribute in an attribute group

All attributes need to be included in attribute groups that can be considered as "tables" of attributes.

Attribute groups are immutable and versioned.

In [ ]:
from snowplow_signals import StreamAttributeGroup, domain_sessionid

attribute_group = StreamAttributeGroup(
    name="my_ecommerce_attributes",
    version=1,
    attribute_key=domain_sessionid,
    attributes=[
        products_added_to_cart,
    ],
    owner="user@company.com",
)

### Applying the attribute group to Signals

The following block pushes the attribute group definition to the Signals API and makes it available for processing.

In [ ]:
published = sp_signals.publish([attribute_group])
print(f"{len(published)} objects published")

### Retrieving data

One can fetch the latest attribute values for a particular session from the attribute group as follow.

In [ ]:
response = attribute_group.get_attributes(
    signals=sp_signals,
    identifier="c5b91072-06b3-4aa7-a558-8e11dcc09993",
)

response